# PREDICTION MODEL FOR S3 AND S4

In [2]:
import pandas as pd 

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import pickle


data=pd.read_csv("/home/user/Downloads/elective/project/yearout/trial.csv")
data.head()
data=data.drop(['student'],axis=1)
def nominalToNumber(dataframe,column,mapping):
             dataframe[column] = dataframe[column].map(mapping)
             return dataframe
def encodeData1(studentData):
             gradeMapping1 = {'O':4,'A+':4,'A':4,'B+':4,'B':4,'C':4,'P':4,'F':0,'FE':0,'I':0,'Absent':0,'not applicable':0}
             gradeMapping = {'O':3,'A+':3,'A':3,'B+':3,'B':3,'C':3,'P':3,'F':0,'FE':0,'I':0,'Absent':0}
             gradeMapping2 = {'O':1,'A+':1,'A':1,'B+':1,'B':1,'C':1,'P':1,'F':0,'FE':0,'I':0,'Absent':0}
             studentData = nominalToNumber(studentData,"MA101",gradeMapping1
                                 )
             studentData = nominalToNumber(studentData,'PH100',gradeMapping1)
             studentData = nominalToNumber(studentData,'BE10105',gradeMapping)
             studentData = nominalToNumber(studentData,'BE110',gradeMapping)
             studentData = nominalToNumber(studentData,'BE103',gradeMapping)
             studentData = nominalToNumber(studentData,'EC100',gradeMapping)
             studentData = nominalToNumber(studentData,'PH110',gradeMapping2)
             studentData = nominalToNumber(studentData,'EC110',gradeMapping2)
             studentData = nominalToNumber(studentData,'CS110',gradeMapping2)
             studentData = nominalToNumber(studentData,"MA102",gradeMapping1
                                 )
             studentData = nominalToNumber(studentData,'CY100',gradeMapping1)
             studentData = nominalToNumber(studentData,'BE100',gradeMapping1)
             studentData = nominalToNumber(studentData,'BE102',gradeMapping)
             studentData = nominalToNumber(studentData,'CY110',gradeMapping2)
             studentData = nominalToNumber(studentData,'EE100',gradeMapping)
             studentData = nominalToNumber(studentData,'EE110',gradeMapping2)
             studentData = nominalToNumber(studentData,'CS100',gradeMapping)
             studentData = nominalToNumber(studentData,'CS120',gradeMapping2)
             studentData = nominalToNumber(studentData,'MA101_supply1',gradeMapping1)
             studentData = nominalToNumber(studentData,'MA101_supply2',gradeMapping1)
             studentData = nominalToNumber(studentData,'PH100_supply1',gradeMapping1)
             studentData = nominalToNumber(studentData,'PH100_supply2',gradeMapping1)
             studentData = nominalToNumber(studentData,'MA102_supply1',gradeMapping1)
             studentData = nominalToNumber(studentData,'MA102_supply2',gradeMapping1)
             studentData = nominalToNumber(studentData,'BE100_supply1',gradeMapping1)
             studentData = nominalToNumber(studentData,'BE100_supply2',gradeMapping1)
             studentData = nominalToNumber(studentData,'CY100_supply1',gradeMapping1)
             studentData = nominalToNumber(studentData,'CY100_supply2',gradeMapping1)
             return studentData
encodeData1(data)  
gradeMapping = {'O':3,'A+':3,'A':3,'B+':3,'B':3,'C':3,'P':3,'F':0,'FE':0,'I':0,'Absent':0,'not applicable':0}
data=data.replace(gradeMapping)
data
subjects=['MA101','PH100','BE10105','BE110','BE103','EC100',"MA102",'CY100','BE100','BE102','EE100','CS100']
for i in subjects:
    n=i
    print(n)
    k=data.columns.get_loc(n)
    internal=k-1
    supply1=k+1
    supply2=k+2
    data['internals']=data.iloc[:,internal]
    data[n]=data.iloc[:,k]
    data['supply1']=data.iloc[:,supply1]
    data['supply2']=data.iloc[:,supply2]
    s=data.loc[:,['tenth','plus_two','keam','internals',n,'supply1','supply2']]
    s[['internals']]=s[['internals']].astype(float)
    a=0
    b=100
    s[['tenth']]=((s[['tenth']]-a)/(b-a))*1
    s[['plus_two']]=((s[['plus_two']]-a)/(b-a))*1
    keam_min_rank=1
    keam_max_rank=50000
    s[['keam']]=1-((s[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*1
    c=0
    d=100
    s[['internals']]=((s[['internals']]-c)/(d-c))*1
    df=s
    df[['internals']]=df[['internals']]*10
    df[[n]]=df[[n]]*4
    df[['supply1']]=df[['supply1']]*3
    df[['supply2']]=df[['supply2']]*2
    
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(df)
    pred=kmeans.predict(df)
    SSE = []
    for cluster in range(1,20):
        kmeans = KMeans(n_jobs = -1, n_clusters = cluster,random_state=16)
        kmeans.fit(data)
        SSE.append(kmeans.inertia_)
    frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
    '''plt.figure(figsize=(12,6))
    plt.title(n)
    plt.plot(frame['Cluster'], frame['SSE'], marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')'''
    kmeans = KMeans(n_jobs = -1, n_clusters = 4,random_state=16)
    kmeans.fit(df)
    pred = kmeans.predict(df)
    frame = pd.DataFrame(df)
    frame['cluster'] = pred
    #print(frame)
    before_cluster=frame.loc[:,['tenth','plus_two','keam','internals','cluster']]
    before_cluster.columns=['tenth','plus_two','keam','internals','cluster']
    if(n=='CY100' or n=='MA102' or n=='BE10105' or n=='BE103' or n=='EE100' or n=='MA101'):
        for i in range(len(before_cluster['cluster'])):
            if(before_cluster['cluster'][i]==0 or before_cluster['cluster'][i]==2 or before_cluster['cluster'][i]==3):
                before_cluster['cluster'][i]=1
            else:
                before_cluster['cluster'][i]=0
    elif(n=='BE102' or n=='CS100' or n=='BE100' or n=='EC100'):
        for i in range(len(before_cluster['cluster'])):
            if(before_cluster['cluster'][i]==0 or before_cluster['cluster'][i]==1 or before_cluster['cluster'][i]==3):
                before_cluster['cluster'][i]=1
            else:
                before_cluster['cluster'][i]=0
    
    elif( n=='BE110' or n=='PH100'):
        for i in range(len(before_cluster['cluster'])):
            if(before_cluster['cluster'][i]==0 or before_cluster['cluster'][i]==1 or before_cluster['cluster'][i]==2):
                before_cluster['cluster'][i]=1
            else:
                before_cluster['cluster'][i]=0
    #print(before_cluster.head(10))
    
    
    x = before_cluster.drop(['cluster'], axis=1)
    y = before_cluster['cluster']
   
    
    train_x,test_x,train_y,test_y = train_test_split(x,y, random_state = 20, stratify=y)
    yearout_supply = KNN(n_neighbors = 8)
    yearout_supply.fit(train_x, train_y)
    test_predict = yearout_supply.predict(test_x)
    k = f1_score(test_predict, test_y)
    print('Test F1 Score    ', k )
    
#filename = 'supply.pkl'
#pickle.dump(yearout_supply, open(filename, 'wb'))

MA101
Test F1 Score     0.962962962962963
PH100
Test F1 Score     0.9655172413793104
BE10105
Test F1 Score     0.9285714285714286
BE110
Test F1 Score     0.9655172413793104
BE103
Test F1 Score     0.9655172413793104
EC100
Test F1 Score     0.8799999999999999
MA102
Test F1 Score     0.9655172413793104
CY100
Test F1 Score     0.9655172413793104
BE100
Test F1 Score     0.9655172413793104
BE102
Test F1 Score     0.9655172413793104
EE100
Test F1 Score     1.0
CS100
Test F1 Score     0.962962962962963


In [4]:
import pandas as pd 

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import pickle


s=pd.read_csv('/home/user/Downloads/elective/project/yearout/s2.csv', sep = ',', engine = 'python', encoding = 'latin-1')
s=s.drop(['student'],axis=1)
def nominalToNumber(dataframe,column,mapping):
    dataframe[column] = dataframe[column].map(mapping)
    return dataframe
def encodeData(studentData):
    gradeMapping1 = {'O':4,'A+':4,'A':4,'B+':4,'B':4,'C':4,'P':4,'F':0,'FE':0,'I':0,'Absent':0}
    gradeMapping = {'O':3,'A+':3,'A':3,'B+':3,'B':3,'C':3,'P':3,'F':0,'FE':0,'I':0,'Absent':0}
    gradeMapping2 = {'O':1,'A+':1,'A':1,'B+':1,'B':1,'C':1,'P':1,'F':0,'FE':0,'I':0,'Absent':0}
    studentData = nominalToNumber(studentData,"MA101",gradeMapping1
                                 )
    studentData = nominalToNumber(studentData,'PH100',gradeMapping)
    studentData = nominalToNumber(studentData,'BE10105',gradeMapping)
    studentData = nominalToNumber(studentData,'BE110',gradeMapping)
    studentData = nominalToNumber(studentData,'BE103',gradeMapping)
    studentData = nominalToNumber(studentData,'EC100',gradeMapping)
    studentData = nominalToNumber(studentData,'PH110',gradeMapping2)
    studentData = nominalToNumber(studentData,'EC110',gradeMapping2)
    studentData = nominalToNumber(studentData,'CS110',gradeMapping2)
    studentData = nominalToNumber(studentData,"MA102",gradeMapping1
                                 )
    studentData = nominalToNumber(studentData,'CY100',gradeMapping1)
    studentData = nominalToNumber(studentData,'BE100',gradeMapping1)
    studentData = nominalToNumber(studentData,'BE102',gradeMapping)
    studentData = nominalToNumber(studentData,'CY110',gradeMapping2)
    studentData = nominalToNumber(studentData,'EE100',gradeMapping)
    studentData = nominalToNumber(studentData,'EE110',gradeMapping2)
    studentData = nominalToNumber(studentData,'CS100',gradeMapping)
    studentData = nominalToNumber(studentData,'CS120',gradeMapping2)   
    return studentData
encodeData(s) 
keam_min_rank=1
keam_max_rank=50000
s[['keam']]=((s[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*100
s[['keam']]=100-(s[['keam']].round(2))
x = s.drop(['yearout'], axis=1)
y = s['yearout']

train_x1,test_x1,train_y1,test_y1 = train_test_split(x,y, random_state = 5)
#print(train_x1)
logreg = LogReg()
logreg.fit(train_x1, train_y1)
train_predict = logreg.predict(train_x1)
k = f1_score(train_predict, train_y1)
print('Training f1_score', k )
test_predict = logreg.predict(test_x1)
k = f1_score(test_predict, test_y1)
print('Test f1_score    ', k )

#filename = 's3_yearout.sav'
#pickle.dump([nominalToNumber, encodeData,logreg] ,open(filename, 'wb'))
         


Training f1_score 1.0
Test f1_score     1.0


# Displaying the working of the model

In [6]:
semester=int(input("enter your current semester:"))
if(semester==3 or semester==4):
    #yearout_supply = pickle.load(open('supply.pkl', 'rb'))
    #logreg=pickle.load(open('s3_yearout.pkl', 'rb'))
    aisse=float(input("enter your 10th marks:"))
    hse=float(input("enter your 12th marks:"))
    keam_rank=float(input("enter your keam rank:"))
    print("\nENTER YOUR GRADES:")
    a=['MA101','PH100','BE10105','BE110','BE103','EC100','PH110','EC110','CS110',"MA102",'CY100','BE100','BE102','CY110','EE100','EE110','CS100','CS120']
    b={}
    k=[]
    d=[]
    for i in a:
        b[i]=input(i+":")
        d.append(b[i])
        if(b[i]=="F" or b[i]=="FE" or b[i]=="Absent"):
            k.append(i)
           
    year=pd.DataFrame(b,index=[0])
    year.insert(0,'tenth',aisse)
    year.insert(1,'plus_two',hse)
    year.insert(2,'keam',keam_rank)
    def nominalToNumber(dataframe,column,mapping):
        dataframe[column] = dataframe[column].map(mapping)
        return dataframe
    def encodeData(studentData):
        gradeMapping1 = {'O':4,'A+':4,'A':4,'B+':4,'B':4,'C':4,'P':4,'F':0,'FE':0,'I':0,'Absent':0}
        gradeMapping = {'O':3,'A+':3,'A':3,'B+':3,'B':3,'C':3,'P':3,'F':0,'FE':0,'I':0,'Absent':0}
        gradeMapping2 = {'O':1,'A+':1,'A':1,'B+':1,'B':1,'C':1,'P':1,'F':0,'FE':0,'I':0,'Absent':0}
        studentData = nominalToNumber(studentData,"MA101",gradeMapping1
                                      )
        studentData = nominalToNumber(studentData,'PH100',gradeMapping)
        studentData = nominalToNumber(studentData,'BE10105',gradeMapping)
        studentData = nominalToNumber(studentData,'BE110',gradeMapping)
        studentData = nominalToNumber(studentData,'BE103',gradeMapping)
        studentData = nominalToNumber(studentData,'EC100',gradeMapping)
        studentData = nominalToNumber(studentData,'PH110',gradeMapping2)
        studentData = nominalToNumber(studentData,'EC110',gradeMapping2)
        studentData = nominalToNumber(studentData,'CS110',gradeMapping2)
        studentData = nominalToNumber(studentData,"MA102",gradeMapping1
                                      )
        studentData = nominalToNumber(studentData,'CY100',gradeMapping1)
        studentData = nominalToNumber(studentData,'BE100',gradeMapping1)
        studentData = nominalToNumber(studentData,'BE102',gradeMapping)
        studentData = nominalToNumber(studentData,'CY110',gradeMapping2)
        studentData = nominalToNumber(studentData,'EE100',gradeMapping)
        studentData = nominalToNumber(studentData,'EE110',gradeMapping2)
        studentData = nominalToNumber(studentData,'CS100',gradeMapping)
        studentData = nominalToNumber(studentData,'CS120',gradeMapping2)   
        return studentData
    
    encodeData(year) 
    keam_min_rank=1
    keam_max_rank=50000
    year[['keam']]=((year[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*100
    year[['keam']]=100-(year[['keam']].round(2))
    
    g={}
    print("enter your internals")
    for j in k:
        g[j]=input(j+":")
        supply=pd.DataFrame(g,index=[0])
        supply.columns=['internals']
        supply.insert(0,'tenth',aisse)
        supply.insert(1,'plus_two',hse)
        supply.insert(2,'keam',keam_rank)
        supply[['internals']]=supply[['internals']].astype(float)
        g.clear()
        a=0
        b=100
        supply[['tenth']]=((supply[['tenth']]-a)/(b-a))*1
        supply[['plus_two']]=((supply[['plus_two']]-a)/(b-a))*1
        keam_min_rank=1
        keam_max_rank=50000
        supply[['keam']]=1-((supply[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*1
        c=0
        d=100
        supply[['internals']]=((supply[['internals']]-c)/(d-c))*1
        supply[['internals']]=supply[['internals']]*10
        
        pred = yearout_supply.predict(supply)
        if(pred==1):
            if(j=='MA101' or j=='PH100' or j=='MA102' or j=='CY100' or j=='BE100' ):
                pred=4
            else:
                pred=3
        else:
            pred=0
        supply['supply']=pred
 
        year[[j]]=supply['supply']
       
        train_predict = logreg.predict(year)
    
    if(train_predict==1):
        print("student won't get year out")
    else:
        print("Student will get yearout")    

enter your current semester:3
enter your 10th marks:87
enter your 12th marks:88
enter your keam rank:5454

ENTER YOUR GRADES:
MA101:A
PH100:A
BE10105:F
BE110:A
BE103:A
EC100:A
PH110:A
EC110:A
CS110:A
MA102:A
CY100:A
BE100:F
BE102:A
CY110:A
EE100:A
EE110:A
CS100:A
CS120:A
enter your internals
BE10105:21
BE100:20
student won't get year out


In [9]:
semester=int(input("enter your current semester:"))
if(semester==3 or semester==4):
    #yearout_supply = pickle.load(open('supply.pkl', 'rb'))
    #logreg=pickle.load(open('s3_yearout.pkl', 'rb'))
    aisse=float(input("enter your 10th marks:"))
    hse=float(input("enter your 12th marks:"))
    keam_rank=float(input("enter your keam rank:"))
    print("\nENTER YOUR GRADES:")
    a=['MA101','PH100','BE10105','BE110','BE103','EC100','PH110','EC110','CS110',"MA102",'CY100','BE100','BE102','CY110','EE100','EE110','CS100','CS120']
    b={}
    k=[]
    d=[]
    for i in a:
        b[i]=input(i+":")
        d.append(b[i])
        if(b[i]=="F" or b[i]=="FE" or b[i]=="Absent"):
            k.append(i)
           
    year=pd.DataFrame(b,index=[0])
    year.insert(0,'tenth',aisse)
    year.insert(1,'plus_two',hse)
    year.insert(2,'keam',keam_rank)
    def nominalToNumber(dataframe,column,mapping):
        dataframe[column] = dataframe[column].map(mapping)
        return dataframe
    def encodeData(studentData):
        gradeMapping1 = {'O':4,'A+':4,'A':4,'B+':4,'B':4,'C':4,'P':4,'F':0,'FE':0,'I':0,'Absent':0}
        gradeMapping = {'O':3,'A+':3,'A':3,'B+':3,'B':3,'C':3,'P':3,'F':0,'FE':0,'I':0,'Absent':0}
        gradeMapping2 = {'O':1,'A+':1,'A':1,'B+':1,'B':1,'C':1,'P':1,'F':0,'FE':0,'I':0,'Absent':0}
        studentData = nominalToNumber(studentData,"MA101",gradeMapping1
                                      )
        studentData = nominalToNumber(studentData,'PH100',gradeMapping)
        studentData = nominalToNumber(studentData,'BE10105',gradeMapping)
        studentData = nominalToNumber(studentData,'BE110',gradeMapping)
        studentData = nominalToNumber(studentData,'BE103',gradeMapping)
        studentData = nominalToNumber(studentData,'EC100',gradeMapping)
        studentData = nominalToNumber(studentData,'PH110',gradeMapping2)
        studentData = nominalToNumber(studentData,'EC110',gradeMapping2)
        studentData = nominalToNumber(studentData,'CS110',gradeMapping2)
        studentData = nominalToNumber(studentData,"MA102",gradeMapping1
                                      )
        studentData = nominalToNumber(studentData,'CY100',gradeMapping1)
        studentData = nominalToNumber(studentData,'BE100',gradeMapping1)
        studentData = nominalToNumber(studentData,'BE102',gradeMapping)
        studentData = nominalToNumber(studentData,'CY110',gradeMapping2)
        studentData = nominalToNumber(studentData,'EE100',gradeMapping)
        studentData = nominalToNumber(studentData,'EE110',gradeMapping2)
        studentData = nominalToNumber(studentData,'CS100',gradeMapping)
        studentData = nominalToNumber(studentData,'CS120',gradeMapping2)   
        return studentData
    
    encodeData(year) 
    keam_min_rank=1
    keam_max_rank=50000
    year[['keam']]=((year[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*100
    year[['keam']]=100-(year[['keam']].round(2))
    
    g={}
    print("enter your internals")
    for j in k:
        g[j]=input(j+":")
        supply=pd.DataFrame(g,index=[0])
        supply.columns=['internals']
        supply.insert(0,'tenth',aisse)
        supply.insert(1,'plus_two',hse)
        supply.insert(2,'keam',keam_rank)
        supply[['internals']]=supply[['internals']].astype(float)
        g.clear()
        a=0
        b=100
        supply[['tenth']]=((supply[['tenth']]-a)/(b-a))*1
        supply[['plus_two']]=((supply[['plus_two']]-a)/(b-a))*1
        keam_min_rank=1
        keam_max_rank=50000
        supply[['keam']]=1-((supply[['keam']]-keam_min_rank)/(keam_max_rank-keam_min_rank))*1
        c=0
        d=100
        supply[['internals']]=((supply[['internals']]-c)/(d-c))*1
        supply[['internals']]=supply[['internals']]*10
        
        pred = yearout_supply.predict(supply)
        if(pred==1):
            if(j=='MA101' or j=='PH100' or j=='MA102' or j=='CY100' or j=='BE100' ):
                pred=4
            else:
                pred=3
        else:
            pred=0
        supply['supply']=pred
 
        year[[j]]=supply['supply']
       
        train_predict = logreg.predict(year)
    
    if(train_predict==1):
        print("student won't get year out")
    else:
        print("Student will get yearout")    

enter your current semester:4
enter your 10th marks:65
enter your 12th marks:64
enter your keam rank:32000

ENTER YOUR GRADES:
MA101:F
PH100:F
BE10105:F
BE110:F
BE103:F
EC100:F
PH110:A
EC110:A
CS110:A
MA102:F
CY100:F
BE100:F
BE102:A
CY110:F
EE100:F
EE110:A
CS100:F
CS120:F
enter your internals
MA101:19
PH100:18
BE10105:17
BE110:20
BE103:19
EC100:18
MA102:17
CY100:16
BE100:20
CY110:20
EE100:19
CS100:19
CS120:18
Student will get yearout
